In [1]:
import pandas as pd
import numpy as np
import json
from log_parser import process_file

In [2]:
with open("../replays/log4.log.rec.json") as f:
    obj = json.load(f)
    
df = pd.DataFrame(obj)
listings = pd.DataFrame(df["listings"][0])
df = df.drop("listings", axis=1)

In [20]:
def flatten_trades(df, col, is_me):
    # get market trades

    market_trades = []
    for index, row  in df.iterrows():
        all_trades = list(row[col].values())
        for sym_trades in all_trades:
            for trade in sym_trades:
                trade["fill_time"] = row["timestamp"]
            market_trades += sym_trades

    market_trades = pd.DataFrame(market_trades)
    market_trades = market_trades.rename({"timestamp": "order_time"}, axis=1)
    
    market_trades["is_me"] = is_me
    market_trades["my_action"] = np.where(market_trades["buyer"] == "SUBMISSION", "buy", None)
    market_trades["my_action"] = np.where(market_trades["seller"] == "SUBMISSION", "sell", None)
    
    return market_trades

In [21]:
market_trades = flatten_trades(
    df, 
    "market_trades", 
    is_me=False
)

my_trades = flatten_trades(
    df, 
    "own_trades", 
    is_me=True
)



In [22]:
trade_df = pd.concat([market_trades, my_trades])
trade_df = trade_df.sort_values(by="fill_time")

In [23]:
trade_df

,buyer,price,quantity,seller,symbol,order_time,fill_time,is_me,my_action
0,,10002.0,1,,PEARLS,100,100,False,None
0,,9998.0,1,SUBMISSION,PEARLS,0,100,True,sell
1,,9996.0,1,SUBMISSION,PEARLS,100,200,True,sell
1,,10002.0,1,,PEARLS,100,200,False,None
2,,10002.0,1,,PEARLS,100,300,False,None
...,...,...,...,...,...,...,...,...,...
1995,,9998.0,7,SUBMISSION,PEARLS,700,199800,True,sell
4624,,4905.0,17,,BANANAS,199500,199800,False,None
4626,,4906.0,3,,BANANAS,199900,199900,False,None
4627,,10002.0,1,,PEARLS,199700,199900,False,None
